## Labels reduction analysis

In [13]:
import pandas as pd
import numpy as np
from seaborn import sns
from scripts import build_dataframe

ModuleNotFoundError: No module named 'seaborn'

DataFrame example

In [8]:
build_dataframe.create_categories_df("../data/products.json", 100).head(8)

,name,description,leaf,max_depth,path,category_0,category_1,category_2,category_3,category_4,category_5,category_6
0,Duracell - AAA Batteries (4-Pack),Compatible with select electronic devices; AAA...,pcmcat303600050001,3,"[pcmcat312300050015, pcmcat248700050021, pcmca...",pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,NaN,NaN,NaN,NaN
1,Duracell - AA 1.5V CopperTop Batteries (4-Pack),Long-lasting energy; DURALOCK Power Preserve t...,pcmcat303600050001,3,"[pcmcat312300050015, pcmcat248700050021, pcmca...",pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,NaN,NaN,NaN,NaN
2,Duracell - AA Batteries (8-Pack),Compatible with select electronic devices; AA ...,pcmcat303600050001,3,"[pcmcat312300050015, pcmcat248700050021, pcmca...",pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,NaN,NaN,NaN,NaN
3,Energizer - MAX Batteries AA (4-Pack),4-pack AA alkaline batteries; battery tester i...,pcmcat303600050001,3,"[pcmcat312300050015, pcmcat248700050021, pcmca...",pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,NaN,NaN,NaN,NaN
4,Duracell - C Batteries (4-Pack),Compatible with select electronic devices; C s...,pcmcat303600050001,3,"[pcmcat312300050015, pcmcat248700050021, pcmca...",pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,NaN,NaN,NaN,NaN
5,Duracell - D Batteries (4-Pack),Compatible with select electronic devices; D s...,pcmcat303600050001,3,"[pcmcat312300050015, pcmcat248700050021, pcmca...",pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,NaN,NaN,NaN,NaN
6,Duracell - 9V Batteries (2-Pack),Compatible with select electronic devices; alk...,pcmcat303600050001,3,"[pcmcat312300050015, pcmcat248700050021, pcmca...",pcmcat312300050015,pcmcat248700050021,pcmcat303600050001,NaN,NaN,NaN,NaN
7,Directed Electronics - Viper Audio Glass Break...,From our expanded online assortment; compatibl...,other,5,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Base function to create DataFrame for analyzing the impact of different thresholds on the total of categories on the dataset

In [9]:
def create_label_reduction_df(json_path: str, thresholds: list):
    # Initialize dataframe
    label_reduction_df = pd.DataFrame(columns=["threshold", "categories_in_df", "percentage_of_categories_used", "avg_category_level", "max_category_level"])
    
    for threshold in thresholds:
        df = build_dataframe.create_categories_df(json_path, threshold)
            
        # Save data for dataframe
        TOT_CATEGORIES = 1864
        categories_in_df = df[build_dataframe.get_category_columns(df)].nunique().sum()
        percentage_of_categories_used = round(categories_in_df / TOT_CATEGORIES  * 100, 2)
        avg_category_level = df["max_depth"].mean()
        max_category_level = np.amax(df["max_depth"].unique())

        # Add row to label_reduction_df
        label_reduction_df.loc[len(label_reduction_df.index)] = [threshold, categories_in_df, percentage_of_categories_used, avg_category_level, max_category_level]
    return label_reduction_df

In [11]:
label_reduction_df = create_label_reduction_df("../data/products.json", [0,1,2,5,10,20,30,50,75,100,150,200,300,500])

In [12]:
label_reduction_df

,threshold,categories_in_df,percentage_of_categories_used,avg_category_level,max_category_level
0,0.0,1864.0,100.00,3.479282,7.0
1,1.0,1589.0,85.25,3.475990,6.0
2,2.0,1452.0,77.90,3.471808,6.0
3,5.0,1193.0,64.00,3.454285,6.0
4,10.0,974.0,52.25,3.425803,6.0
5,20.0,764.0,40.99,3.368993,6.0
6,30.0,623.0,33.42,3.299926,6.0
7,50.0,438.0,23.50,3.168029,6.0
8,75.0,335.0,17.97,3.045560,5.0
9,100.0,268.0,14.38,2.943171,5.0


In [ ]:
sns.lineplot(y=label_reduction_df.threshold, x = label_reduction_df.categories_in_df)

In [ ]:
sns.lineplot(y=label_reduction_df.threshold, x = label_reduction_df.avg_category_level)